In [1]:
import json
import Monster

# Formatting Tests

When you bulk export your monster maker monsters, it appears to export a "vault". Every monster is part of the "vault" and, from what I can tell, the blueprint is the actual modified statblock of the creature so we'll need

```
for monster in mm_json["vault"]:
    monster=monster["blueprint"]
```
my plan is then to create a monster object with one property per key in the roll20 JSON. Below, I've started to fill out as many basic properties of the monster as I can by comparing the two jsons.



In [2]:
mm_vault_json=open("monster_vault Template.json")
mm_vault_json=json.load(mm_vault_json)

In [3]:
r20_json=open("Roll20_Lich.json")
r20_json=json.load(r20_json)

In [4]:
mm_complex_json=open("complex_monster_MM.json")
mm_complex_json=json.load(mm_complex_json)

In [5]:
monster=mm_vault_json["vault"][1]["blueprint"]
complex_monster=mm_complex_json["blueprint"]

In [6]:
custom_monster=open("complex_monster_MM_custom.json")
custom_monster=json.load(custom_monster)
custom_monster=custom_monster["blueprint"]

In [ ]:
monster=custom_monster.copy()

name=monster["description"]["name"]
size=monster["description"]["size"]
monster_type=monster["description"]["type"]
alignment=monster["description"]["alignment"]
hp=f"{monster['hp']['average']} ({monster['hp']['roll']})"
cr=monster["challenge"]["rating"]

#how do I get this?
proficiency=1

#get AC from base+mod then make a string with the AC type
if monster["ac"]["modifier"]==None:
    ac=0.0
else:
    ac=float(monster["ac"]["modifier"])
ac=monster["ac"]["base"]+ac
ac=f"{ac} ({monster['ac']['type']})"


#speed and senses are dictionaries of type:distance
#so we combine to a comma separated list of type and distance
speed=monster["speed"]
speed=", ".join([f"{x.capitalize()} {y}" for x,y in speed.items()])

senses=monster["senses"]
senses=", ".join([f"{x.capitalize()} {y}" for x,y in senses.items() if y is not None])

languages=", ".join([x["name"].capitalize() for x in monster["languages"]])

abilities=dict(zip(list(monster["abilities"].keys())[:6],list(monster["abilities"].values())[:6]))

skills=[]
for skill in monster["skills"]:
    skill_name=skill["name"].capitalize()
    skill_mod=abilities[skill["custom"]["ability"]]+proficiency
    skills.append(f"{skill_name} +{skill_mod}")
skills=", ".join(skills)

resistances=", ".join([f"{x['type'].capitalize()}" for x in monster["resistances"]])
immunities=", ".join([f"{x['type'].capitalize()}" for x in monster["immunities"]])
vulnerabilities=", ".join([f"{x['type'].capitalize()}" for x in monster["vulnerabilities"]])
conditions=", ".join([f"{x['type'].capitalize()}" for x in monster["conditions"]])

#traits in both formats are a list of dictionaries with name and description as keys
#however description has a different name in each format so loop to rename
traits=monster["traits"]
for i in range(len(traits)):
    trait=traits[i]
    trait["text"]=trait.pop("detail","")
    traits[i]=trait
    
#Same deal for actions
actions=monster["actions"]
for i in range(len(actions)):
    action=actions[i]
    action["text"]=action.pop("detail","")
    actions[i]=action
    
#and reactions
reactions=monster["reactions"]
for i in range(len(actions)):
    action=reactions[i]
    action["text"]=action.pop("detail","")
    reactions[i]=action
    
# finally legendary actions
# MM has no concept of cost so it will just be in the detail
legendaryActions=monster["legendaryActions"]
for i in range(len(actions)):
    action=legendaryActions[i]
    action["text"]=action.pop("detail","")
    legendaryActions[i]=action
    
legendaryPoints=monster['legendaryActionsPerRound']

# Class tests
Once we have the monster stored in a class we can look at how to make a roll20 dict.

In [7]:
r20_json.keys()

dict_keys(['name', 'size', 'type', 'alignment', 'AC', 'HP', 'speed', 'strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma', 'savingThrows', 'skills', 'damageResistances', 'damageImmunities', 'conditionImmunities', 'senses', 'languages', 'challenge', 'traits', 'actions', 'reactions', 'legendaryPoints', 'legendaryActions'])

In [8]:
custom_monster=Monster.Monster(custom_monster)


In [9]:
custom_monster.to_r20_format()

'{"name": "Mich", "size": "medium", "alignment": "unaligned", "challenge": "5", "proficiency": 1, "speed": "Normal 30 ft., Burrow 30 ft., Climb None, Fly None, Swim None, Other None", "senses": "Blindsight 30 ft., Darkvision 30 ft.", "languages": "Common", "savingThrows": "", "skills": "Acrobatics +19, Religion +19", "damageResistances": "Fire, Force, Acid", "damageImmunities": "Fire, Acid", "vulnerabilities": "Acid, Slashing, Psychic", "conditionImmunities": "Blinded, Unconscious", "traits": [{"name": "(Striker) Savage Assault", "text": "Once per turn, add your level in extra damage to an attack."}, {"name": "Shifty", "text": "You can _Disengage_ as a bonus action."}], "actions": [{"name": "Bash", "text": "_Melee Weapon Attack:_ [attack] vs AC. _Hit:_ [damage, d4] slashing damage."}, {"name": "Kill", "text": "_Melee Weapon Attack:_ [dc-primary] vs Strength. _Hit:_ the target is pushed up to 10 ft away."}], "reactions": [{"name": "cry", "text": "you hurt me"}, {"name": "why?", "text": 

In [ ]:
out_dict=vars(custom_monster)
out_dict["type"]=out_dict.pop("monster_type","")

for key in ["hp","ac"]:
    out_dict[key.upper()]=out_dict.pop(key,"")
    
ability_keys=list(out_dict["abilities"].keys())
for i,key in enumerate(['strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma']):
    out_dict[key]=out_dict["abilities"][ability_keys[i]]
deleted=out_dict.pop("abilities","")

In [ ]:
for key in r20_json.keys():
    print(out_dict[key],"|",r20_json[key])

In [ ]:
f=open("r20_test.json","w")
json.dump(out_dict,f)

In [ ]:
f.close()